Below is a Jupyter Notebook-style tutorial that demonstrates how to plot seismic event data on a map using PyGMT. The tutorial covers reading seismic data from a CSV file, configuring a region of interest, plotting topography, and visualizing earthquake events with color-coding based on depth.

In [2]:
# seismic_event_mapping.ipynb

# Mapping Seismic Events with PyGMT
# =================================

# **Objective**: This notebook demonstrates how to visualize seismic event data on a map using the `PyGMT` library. 
# The tutorial includes reading data from a CSV file, defining a region of interest, plotting topography, and 
# displaying earthquake events with markers sized according to magnitude and color-coded by depth.

# Importing Necessary Libraries
# -----------------------------

# We'll begin by importing the necessary libraries. `pandas` will be used for data manipulation, 
# and `pygmt` will be used for creating the map.

import pandas as pd
import pygmt

Reading the Seismic Event Data
------------------------------
The seismic event data, which includes information such as latitude, longitude, magnitude, and depth,
will be read from a CSV file using pandas.

In [3]:
# Read the CSV file containing seismic event data
df = pd.read_csv("./seismic_events.csv")

Defining the Region of Interest
-------------------------------
To ensure that the map encompasses all seismic events, we'll determine the minimum and maximum latitude
and longitude from the data. We'll extend the boundaries slightly to provide a margin around the events.

In [4]:
# Determine the min and max latitude and longitude
min_lat = df["Latitude"].min() - 0.1  # Subtract 0.1 degree for margin
max_lat = df["Latitude"].max() + 0.1  # Add 0.1 degree for margin
min_lon = df["Longitude"].min() - 0.1  # Subtract 0.1 degree for margin
max_lon = df["Longitude"].max() + 0.1  # Add 0.1 degree for margin
region = [min_lon, max_lon, min_lat, max_lat]  # [west, east, south, north]

Creating the Base Map with Topography
-------------------------------------
We will use the pygmt.grdimage function to create a base map that includes topography. The @earth_relief_30s
dataset provides global topographic data at 30 arc-second resolution.

In [6]:
# Define the topography data source
topo_data = '@earth_relief_30s'

# Create a figure object and plot the topography
fig = pygmt.Figure()
fig.grdimage(grid=topo_data, cmap="white", region=region, projection='M6i', shading=True, frame=True)

Customizing the Color Scale for Depth
-------------------------------------
We'll use a continuous color palette (cpt) to represent the depth of the earthquakes.
The color scale will be adjusted according to the minimum and maximum depths in the dataset.

In [7]:
# Create a color palette for depth
pygmt.makecpt(cmap="cool", series=[df.Depth.min(), df.Depth.max()], continuous=True)

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


Plotting Earthquake Events
--------------------------
Each earthquake event will be plotted on the map using the fig.plot function.
The marker size will be proportional to the magnitude, and the color will represent the depth.

In [8]:
# Plot the earthquake events with marker size proportional to magnitude and color by depth
fig.plot(x=df.Longitude, y=df.Latitude, 
         size=0.02*(2 ** df.Magnitude),  # Size proportional to magnitude
         fill=df.Depth,  # Color by depth
         cmap=True, 
         style="cc",  # Circular markers
         pen="black")  # Black outline for markers

# Add a colorbar to show the depth scale
fig.colorbar(frame='af+lFocal Depth (km)')

Displaying the Map
------------------
Finally, we'll display the map. If needed, the map can also be saved to a file for later use.

In [10]:
# Show the map
fig.show()

# Save the figure to a file if needed
# fig.savefig("isc_events_map.png")